In [17]:
from operator import add
sc = SparkContext.getOrCreate()

In [18]:
diretorioBase = os.path.join('data')
caminho = os.path.join('web-Google.txt')
arquivo = os.path.join(diretorioBase, caminho)

In [19]:
def atualizaRank(listaUrls, rank):
    num_urls = len(listaUrls)
    rankAtualizado = []
    for x in listaUrls:
        rankAtualizado.append((x, rank / num_urls))
    return rankAtualizado

In [20]:
#numPartitions = 2
#rawData = sc.textFile(fileName, numPartitions)
linksGoogle = sc.textFile(arquivo).filter(lambda x: "#" not in x).map(lambda x: x.split("\t"))

In [21]:
#print linksGoogle.count()

In [22]:
linksAgrupados = linksGoogle.groupByKey().cache()
#print(linksAgrupados.take(1))
#for it in linksAgrupados.take(1)[0][1]:
#    print(it)


In [23]:
ranks = linksAgrupados.map(lambda url_agrupados: (url_agrupados[0], 1.0))

In [24]:
for x in range(2):
    # Adiciona ranks inicializados com 1.0 na posição [1][1] da matriz
    agrupaIdLinkComRank = linksAgrupados.join(ranks)
    rankAtualizado = agrupaIdLinkComRank.flatMap(lambda url_rank: atualizaRank(url_rank[1][0], url_rank[1][1]))
    # Soma os valores com o mesmo id
    ranks = rankAtualizado.reduceByKey(lambda x,y: x+y).mapValues(lambda rankFatorD: rankFatorD * 0.85 + 0.15)

for (link, rank) in ranks.sortBy(lambda x:-x[1]).take(10):
    print("O id do link %s  tem o ranking: %s." % (link, rank))

O id do link 163075  tem o ranking: 639.28154451.
O id do link 597621  tem o ranking: 625.461060004.
O id do link 537039  tem o ranking: 607.916883693.
O id do link 885605  tem o ranking: 595.209230633.
O id do link 41909  tem o ranking: 583.989571807.
O id do link 837478  tem o ranking: 573.317608333.
O id do link 551829  tem o ranking: 551.389531126.
O id do link 605856  tem o ranking: 551.330580279.
O id do link 504140  tem o ranking: 537.373879516.
O id do link 407610  tem o ranking: 492.568315758.
